In [4]:
import os
import pandas as pd
import numpy as np
os.chdir('/home/peisenha/restudToolbox/package/development/testing/property')



In [159]:
data_frame = pd.read_pickle('debug.respy.pkl')



In [187]:
(data_frame['Choice'].loc[slice(None), 1].isin([1, 2])) 

Identifier
0.0       True
1.0       True
2.0       True
3.0       True
4.0       True
5.0       True
6.0       True
7.0      False
8.0       True
9.0       True
10.0      True
11.0      True
12.0      True
13.0      True
14.0      True
15.0      True
16.0      True
17.0     False
18.0     False
19.0      True
20.0      True
21.0     False
22.0      True
23.0      True
24.0      True
25.0      True
26.0     False
27.0      True
28.0      True
29.0      True
         ...  
822.0     True
823.0     True
824.0     True
825.0     True
826.0     True
827.0     True
828.0     True
829.0    False
830.0     True
831.0     True
832.0     True
833.0     True
834.0     True
835.0     True
836.0     True
837.0     True
838.0     True
839.0     True
840.0     True
841.0     True
842.0     True
843.0     True
844.0     True
845.0     True
846.0     True
847.0     True
848.0    False
849.0     True
850.0     True
851.0     True
Name: Choice, dtype: bool